### Here we collect test target

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.window import Window as w
import pandas as pd
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings("ignore")

spark = (
    SparkSession.builder
    .config("spark.driver.memory","100g")
    .config("spark.hadoop.fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:3.2.2")
    .config("fs.s3a.aws.credentials.provider","com.amazonaws.auth.DefaultAWSCredentialsProviderChain")
    .config("spark.driver.maxResultSize",0)
    # .config('spark.hadoop.io.compression.codecs', 'nl.basjes.hadoop.io.compress.SplittableGzipCodec')
    .getOrCreate()
)
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark.conf.set("spark.hadoop.io.compression.codecs", "org.apache.hadoop.io.compress.BZip2Codec")

s3_bucket_spark = "s3a://820323602090-team-dbad373c-7e36-407b-8690-05a44b804f43"
s3_bucket_pandas = "s3a://820323602090-team-dbad373c-7e36-407b-8690-05a44b804f43"

:: loading settings :: url = jar:file:/home/jovyan/.conda/envs/python3/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-607cf2d0-f15b-429d-ab82-5dfbbca521ef;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 148ms :: artifacts dl 4ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	--------------------------------

In [2]:
!aws s3 ls $team_bucket/data/raw/

                           PRE DIM_DIAGNOSIS/
                           PRE DIM_DRUG/
                           PRE DIM_HCPT/
                           PRE DX_DIAGNOSIS/
                           PRE MEDICAL_HEADERS/
                           PRE MEDICAL_SERVICE_LINES/
                           PRE PATIENT_ENROLLMENTS/
                           PRE PATIENT_SUMMARIES/
                           PRE PAYERS/
                           PRE PHARMACY/
                           PRE PROVIDER_SUMMARIES/
                           PRE VISIT_SUMMARIES/


In [3]:
df = spark.read.parquet(s3_bucket_spark+"/data/raw/PROVIDER_SUMMARIES/")

23/04/22 12:37:36 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [4]:
df.columns

['VALUE',
 'COHORT_ID',
 'FIRST_NAME',
 'MIDDLE_NAME',
 'LAST_NAME',
 'SUFFIX',
 'DEACTIVATION_DATE',
 'REACTIVATION_DATE',
 'PRIMARY_HCO_ID',
 'PROCESS_DATE',
 'PROVIDER_ID',
 'PROVIDER_STATE',
 'PROVIDER_TYPE',
 'SPECIALTY1',
 'SPECIALTY2',
 'CREDENTIAL_TYPE',
 'KCS1',
 'KCS2']

In [7]:
df.select('VALUE', 'PROVIDER_ID', 'SPECIALTY1', 'SPECIALTY2').show(5)

+--------------------+-----------+-----------------+----------------+
|               VALUE|PROVIDER_ID|       SPECIALTY1|      SPECIALTY2|
+--------------------+-----------+-----------------+----------------+
|{"COHORT_ID":1043...| 1952599888|Assistive Therapy|            null|
|{"COHORT_ID":1043...| 1952599904|       Podiatrist|            null|
|{"COHORT_ID":1043...| 1952599912|          Dentist|            null|
|{"COHORT_ID":1043...| 1952599938|       Psychiatry|General Practice|
|{"COHORT_ID":1043...| 1952599946|  Behavioral Care|            null|
+--------------------+-----------+-----------------+----------------+
only showing top 5 rows



In [10]:
submission = spark.read.parquet('../submission/submission_team_xxx.parquet')

In [11]:
submission.show(5)

+------------+-----------+
|PROVIDER_NPI|PROBABILITY|
+------------+-----------+
|  1710065842|          0|
|  1700128584|          0|
|  1265434021|          0|
|  1598178329|          0|
|  1598129975|          0|
+------------+-----------+
only showing top 5 rows



In [16]:
test_target = submission.select('PROVIDER_NPI').join(
    df.select('PROVIDER_ID', 'SPECIALTY2').withColumnRenamed('PROVIDER_ID', 'PROVIDER_NPI'), how='left', on='PROVIDER_NPI'
)

In [17]:
test_target.show(5)

+------------+-----------------+
|PROVIDER_NPI|       SPECIALTY2|
+------------+-----------------+
|  1265434021|             null|
|  1598178329|Internal Medicine|
|  1700128584| Surgery, General|
|  1922045095| General Practice|
|  1598129975|             null|
+------------+-----------------+
only showing top 5 rows



In [18]:
test_target.select('PROVIDER_NPI').distinct().count()

13416

In [20]:
test_target.write.parquet('../data/02_intermediate/test_target.parquet')

In [22]:
spark.stop()